# [`tf.placeholder`](https://www.tensorflow.org/api_docs/python/tf/placeholder)

* `tf.Session()` 을 실행 할 때 외부에서 값을 넣어줌
* 학습데이터 또는 추론(inference) 할 때의 개별 데이터처럼 그래프 외부에서 값을 넣어주는 형태로 만들 필요가 있을 때 유용함

In [ ]:
import tensorflow as tf

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

tf.set_random_seed(219)

### `tf.placeholder`

* `tf.Session()`을 열어서 실행할 때 값을 넣어줘야 한다.
* 아래 예제처럼 그냥 `tf.Session()`을 실행하면 error가 생긴다.

In [ ]:
# create a placeholder
a = tf.placeholder(tf.float32, shape=[3])

# create a constant of type
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b  # Short for tf.add(a, b)

with tf.Session(config=sess_config) as sess:
  print(sess.run(c)) # Error because a doesn’t have any

### `tf.placeholder` 올바른 예제

* `sess.run`할 때 `feed_dict`이라는 인자를 사용하여 placeholder `a`의 실제 값을 넣어준다.
* `c`를 계산하려면 `c`에 연결된 모든 computational graph의 변수가 필요하다.
  * 여기서는 `a`와 `b`의 값이 필요하다.
  * `a`는 `tf.placeholder`형태이므로 `sess.run`에 `feed_dict`형태로 실제 값을 넣어줘야한다.
  * `x`는 전체 그래프에는 속하지만 `c`와 직접적인 연결(`c`를 계산하는데 `x`가 필요없음)이 없으므로 `feed_dict`에는 필요없다.

In [ ]:
# create a placeholder
a = tf.placeholder(tf.float32, shape=[3])
# create unused placeholder
x = tf.placeholder(tf.float32, shape=[3])

# create a constant of type
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b  # Short for tf.add(a, b)

with tf.Session(config=sess_config) as sess:
  print(sess.run(c, feed_dict={a: [1, 2, 3]}))

### Normal Loading

* Graph 구성부분과 Session 부분을 완벽하게 분리해야한다. (정상적인 방법)

In [ ]:
# Only necessary if you use IDLE or a jupyter notebook
tf.reset_default_graph()

x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) # you create the node for add node before executing the graph

with tf.Session(config=sess_config) as sess:
  sess.run(tf.global_variables_initializer())
  writer = tf.summary.FileWriter('./graphs/06_normal', sess.graph)
  for _ in range(10):
    print(sess.run(z))
  writer.close()
  print('\n')

  for node in tf.get_default_graph().as_graph_def().node:
    print(node.name)

### Lazy Loading

* Graph 구성부분과 Session 부분을 완벽하게 분리하지 않았다.
  * `tf.add`가 `for`문 안에서 계속 생성이 되는 구조
  * `session`을 실행할 때 마다 새로운 노드가 추가됨
  * 결과값에 에러는 없지만 실제로는 잘못된 코드

In [ ]:
# Only necessary if you use IDLE or a jupyter notebook
tf.reset_default_graph()

x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

with tf.Session(config=sess_config) as sess:
  sess.run(tf.global_variables_initializer())
  writer = tf.summary.FileWriter('./graphs/06_lazy', sess.graph)
  for _ in range(10):
    print(sess.run(tf.add(x, y)))
  writer.close()
  print('\n')
  
  for node in tf.get_default_graph().as_graph_def().node:
    print(node.name)